Attempted to push results to github but it resulted in filesize being too big (>100MB). Therefore, will try to save to drive instead, as downloading the file is ineffective. Github Credential Helper is likely redundant.

Idea: zip everything but the weights, as they are the only thing really using storage. The rest can quickly be downloaded and evaluated, independently. Weights also need to be obtained, so drive is still wanted.

In [1]:
import os
HOME = '/content'

In [2]:
WEIGHTS_RELEASE_VERSION = 'v1.0'
WEIGHTS_NAME = 'yolov9-e'
DATASET_NAME = 'FIMUS-train'

In [3]:
from google.colab import drive
DRIVE = '/content/drive'
drive.mount(DRIVE)
DRIVE = DRIVE+'/MyDrive'

Mounted at /content/drive


In [4]:
!nvidia-smi

Mon Apr 29 07:13:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Install yolov9 and pretrained weights

In [5]:
!git clone https://github.com/Hallvaeb/yolov9-masterthesis.git

Cloning into 'yolov9-masterthesis'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 155 (delta 46), reused 141 (delta 37), pack-reused 0
Receiving objects: 100% (155/155), 580.67 KiB | 34.16 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [6]:
%cd yolov9-masterthesis
!pip install -r requirements.txt -q

/content/yolov9-masterthesis
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.9 MB/s eta 0:00:00


In [7]:
!wget -P {HOME}/weights -q https://github.com/Hallvaeb/yolov9-masterthesis/releases/download/{WEIGHTS_RELEASE_VERSION}/{WEIGHTS_NAME}.pt

## Download and prepare the dataset for training

### Download dataset

In [8]:
!mkdir -p {HOME}/dataset
%cd {HOME}/dataset

/content


In [9]:
!gdown https://drive.google.com/uc?id=1fWFNpGyk8FXtYqE1FJplOwylEhoPVx8h

Downloading...
From (original): https://drive.google.com/uc?id=134QOvaatwKdy0iIeNqA_p-xkAhkV4F8Y
From (redirected): https://drive.google.com/uc?id=134QOvaatwKdy0iIeNqA_p-xkAhkV4F8Y&confirm=t&uuid=9d5b4eb3-623f-487e-9763-4265fd9a11b0
To: /content/CrowdHuman_train01.zip
100% 2.97G/2.97G [00:54<00:00, 54.7MB/s]


In [18]:
!unzip -n -q {HOME}/{DATASET_NAME}.zip -d {HOME}/dataset

## Train the model
Prior to running this, /models/detect/yolov9-e.yaml was modified to have nc: 1. This file is responsible for the detector architecture. We freeze the backbone, to shorten training time.

In [ ]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch -1 --epochs 5 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/yolov9-masterthesis/data.yaml \
--weights /content/weights/yolov9-e.pt \
--cfg yolov9-e.yaml \
--freeze 10 \
--project . \
--single-cls

/content
2024-04-29 07:37:44.972777: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 07:37:44.972829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 07:37:44.974182: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-29 07:37:46.011857: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train_dual: weights=/content/weights/yolov9-e.pt, cfg=yolov9-e.yaml, data=/content/yolov9-masterthesis/data.yaml, hyp=yolov9-masterthesis/data/hyps/hyp.scratch-high.yaml, epochs=50, batch_size=-1, imgsz=640, rect=False, resume=False

In [ ]:
results_saved_to = "exp"

from datetime import datetime
timestamp = datetime.now().strftime("%d%m%y-%H%M%S")

!zip -r {DRIVE}/{timestamp}.zip {HOME}/$results_saved_to